In [4]:
!.venv/bin/pip3 install -r requirements.txt

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try apt install
    python3-xyz, where xyz is the package you are trying to
    install.
    
    If you wish to install a non-Debian-packaged Python package,
    create a virtual environment using python3 -m venv path/to/venv.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip. Make
    sure you have python3-full installed.
    
    If you wish to install a non-Debian packaged Python application,
    it may be easiest to use pipx install xyz, which will manage a
    virtual environment for you. Make sure you have pipx installed.
    
    See /usr/share/doc/python3.12/README.venv for more information.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detai

In [1]:
import pandas as pd
import numpy as np
import tensorflow.keras.layers
from keras.src.layers import Lambda
from matplotlib import pyplot as plt
import seaborn as sns
import tensorflow as tf
import math
import tensorflow_decision_forests as tfdf
import ydf  # Yggdrasil Decision Forests
from sklearn.model_selection import train_test_split
from wurlitzer import sys_pipes
import keras.layers as preprocessing
from sklearn.model_selection import KFold
import keras_tuner as kt
import tqdm as tqdm

%matplotlib inline
plt.rcParams['figure.figsize'] = [16, 10]

2025-03-28 18:25:19.643785: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743200719.654857    9376 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743200719.658410    9376 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1743200719.668039    9376 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743200719.668046    9376 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743200719.668048    9376 computation_placer.cc:177] computation placer alr

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Set memory growth to avoid TensorFlow using all GPU memory
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)

        # Set TensorFlow to use only the first GPU (if multiple GPUs available)
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')

        print("Using GPU:", gpus[0])
    except RuntimeError as e:
        print(e)

Using GPU: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [3]:
def train_and_eval(model, train_ds, test_ds = None):
    # Optionally, add evaluation metrics.
    model.compile(metrics=["mse"])
    rmse = 0

    with sys_pipes():
        model.fit(x=train_ds)

    if test_ds is not None:
        evaluation = model.evaluate(x=test_ds, return_dict=True)
        rmse = math.sqrt(evaluation["mse"])

    return rmse

def latlon_to_xyz(lat, lon):
    lat, lon = np.radians(lat), np.radians(lon)
    x = np.cos(lat) * np.cos(lon)
    y = np.cos(lat) * np.sin(lon)
    z = np.sin(lat)
    return x, y, z

# Example: Normalize Cartesian coordinates between 0 and 1
def normalize_xyz(x, y, z):
    # Normalizing each coordinate to the [0, 1] range
    return (x + 1) / 2, (y + 1) / 2, (z + 1) / 2

class LatLonToXYZ(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(LatLonToXYZ, self).__init__(**kwargs)

    def call(self, inputs):
        lat, lon = inputs[..., 0], inputs[..., 1]  # Assuming inputs shape is (..., 2)
        rad_factor = tf.constant(np.pi / 180, dtype=tf.float32)
        lat, lon = lat * rad_factor, lon * rad_factor

        x = tf.cos(lat) * tf.cos(lon)
        y = tf.cos(lat) * tf.sin(lon)
        z = tf.sin(lat)

        return tf.stack([x, y, z], axis=-1)

class NormalizeXYZ(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(NormalizeXYZ, self).__init__(**kwargs)

    def call(self, inputs):
        return (inputs + 1) / 2  # Normalize to [0,1] range

def get_category_encoding_layer(name, dataset, dtype, max_tokens=None):
    # Create a StringLookup layer which will turn strings into integer indices
    if dtype == 'string':
        index = preprocessing.StringLookup(max_tokens=max_tokens)
    else:
        index = preprocessing.IntegerLookup(max_tokens=max_tokens)

    # TODO
    # Prepare a Dataset that only yields our feature
    feature_ds = dataset.map(lambda x, y: tf.compat.as_str_any(x[name]))

    # Learn the set of possible values and assign them a fixed integer index.
    index.adapt(feature_ds)

    # Create a Discretization for our integer indices.
    encoder = preprocessing.CategoryEncoding(num_tokens=index.vocabulary_size())

    # Apply one-hot encoding to our indices. The lambda function captures the
    # layer so we can use them, or include them in the functional model later.
    # return lambda feature: encoder(index(feature))

    return Lambda(lambda feature: encoder(index(feature)))

def dataset_preprocessing(unused_columns, categorical_ft_columns, X_train):
    used_columns = []

    raw_inputs = {}
    processed_inputs = {}

    for col in X_train.columns:
        dtype = X_train.dtypes[col]
        if dtype == "object":
            dtype = "string"
        raw_inputs[col] = tf.keras.layers.Input(shape=(1,), name=col, dtype=dtype)
        processed_inputs[col] = raw_inputs[col]

    processed_inputs["phi"] = raw_inputs["total_individuals"] / raw_inputs["total_households"]
    processed_inputs["id_area"] = raw_inputs["total_individuals"] / raw_inputs["AREA_SQKM"]
    processed_inputs["hs_area"] = raw_inputs["total_households"] / raw_inputs["AREA_SQKM"]
    lon_lat = tf.keras.layers.Concatenate(name="lon_lat")([raw_inputs["lon"], raw_inputs["lat"]])
    processed_inputs["xyz"] = NormalizeXYZ()(LatLonToXYZ()(lon_lat))
    zone_encoder = get_category_encoding_layer("ADM2_ID", train_ds, "string", 100)
    processed_inputs["zone"] = zone_encoder(raw_inputs["ADM2_ID"])
    processed_cols = ["total_individuals", "total_households", "AREA_SQKM", "lon", "lat", "ADM2_ID"]
    new_cols = ["phi", "id_area", "hs_area", "xyz", "zone"]

    all_features = []
    for col in X_train.columns:
        if col not in unused_columns + processed_cols:
            if col in categorical_ft_columns:
                all_features.append(tfdf.keras.FeatureUsage(name=col, semantic=tfdf.keras.FeatureSemantic.CATEGORICAL))
            else:
                all_features.append(tfdf.keras.FeatureUsage(name=col))
            used_columns.append(col)

    for col in new_cols:
        all_features.append(tfdf.keras.FeatureUsage(name=col))
        used_columns.append(col)

    preprocessor = tf.keras.Model(inputs=raw_inputs, outputs=processed_inputs, name="preprocessor")

    return all_features, preprocessor

In [4]:
ROOT_DIR = "temporary"
df = pd.read_csv(f'{ROOT_DIR}/Train.csv')
test_df = pd.read_csv(f'{ROOT_DIR}/Test.csv')
vocab_df = pd.read_csv(f'{ROOT_DIR}/variable_descriptions.csv')
admin_df = pd.read_csv(f'{ROOT_DIR}/zaf_adminboundaries_tabulardata.csv', sep=";")

admin_df = admin_df[["ADM4_PCODE", "AREA_SQKM", "ADM2_ID"]] # ADM3_ID
admin_df["AREA_SQKM"] = admin_df["AREA_SQKM"].str.replace(",", ".").astype(float)
df = pd.merge(df, admin_df, on="ADM4_PCODE", how="left")
test_df = pd.merge(test_df, admin_df, on="ADM4_PCODE", how="left")

X = df.drop("target", axis=1)
y = df["target"]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=False)

train_ds_pd = pd.concat([X_train, y_train], axis=1)
test_ds_pd = pd.concat([X_val, y_val], axis=1)
label_column = "target"
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label=label_column, task=tfdf.keras.Task.REGRESSION)
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_ds_pd, label=label_column, task=tfdf.keras.Task.REGRESSION)

default_columns = ["ward", "ADM4_PCODE"]
nn_cols = ["dw_00", "psa_00", "lan_00", "pg_00", "pw_00", "stv_00", "car_00", "lln_00"]
categorical_ft_columns = ["ADM2_ID"]

ft_columns = [
    default_columns + ["dw_12", "dw_13", "lan_13", "pw_08", "pw_07"],
    default_columns + ["dw_12", "dw_13", "lan_13", "pw_08", "pw_07"] + nn_cols,
    default_columns,
]

I0000 00:00:1743200994.524158    9376 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22178 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:01:00.0, compute capability: 8.9


In [ ]:
all_models = {
    "RF": tfdf.keras.RandomForestModel,
    "GBM": tfdf.keras.GradientBoostedTreesModel
}
unused_columns = ft_columns[0]
preprocess_function = dataset_preprocessing
tuning_logs = {mn: {} for mn in all_models}

n_fold = 10
kf = KFold(n_splits=n_fold, shuffle=False)
hps = []
rmse = []
# Find best HP
for train_index, test_index in tqdm.tqdm(kf.split(X_train), desc="Tuning"):
    X_fold_train, X_fold_test = X_train.iloc[train_index], X_train.iloc[test_index]
    y_fold_train, y_fold_test = y_train.iloc[train_index], y_train.iloc[test_index]

    X_fold_train["target"] = y_fold_train
    X_fold_test["target"] = y_fold_test
    train_fold = tfdf.keras.pd_dataframe_to_tf_dataset(X_fold_train, label=label_column, task=tfdf.keras.Task.REGRESSION)
    test_fold = tfdf.keras.pd_dataframe_to_tf_dataset(X_fold_test, label=label_column, task=tfdf.keras.Task.REGRESSION)

    for mn, md in all_models.items():
        tuner = tfdf.tuner.RandomSearch(num_trials=10, use_predefined_hps=True)
        all_features, preprocessor = preprocess_function(unused_columns, categorical_ft_columns, X_train)
        tuned_model = md(features=all_features, exclude_non_specified_features=True, preprocessing=preprocessor, task=tfdf.keras.Task.REGRESSION, tuner=tuner)
        tuned_model.compile(metrics=["mse"])
        tuned_model.fit(train_ds)

        rmse = math.sqrt(tuned_model.evaluate(test_ds, return_dict=True, verbose=0)["mse"])
        tuning_logs = tuned_model.make_inspector().tuning_logs()
        hp = tuning_logs[tuning_logs.best].iloc[0]

        model_log = tuning_logs.get(mn, {})
        tuning_logs[mn] = {
            "rmse": model_log.get("rmse", []) + [rmse],
            "hp": model_log.get("hp", []) + [hp],
        }

# Eval with best HP
for mn, logs in tuning_logs.items():
    mean_rmse = np.mean(logs["rmse"])
    std_rmse = np.std(logs["rmse"])
    best_hp_idx = np.argmin(logs["hp"])
    md = all_models[mn]
    best_hp = logs["hp"][best_hp_idx]
    all_features, preprocessor = preprocess_function(unused_columns, categorical_ft_columns, X_train)
    best_model = md(features=all_features, exclude_non_specified_features=True, preprocessing=preprocessor, task=tfdf.keras.Task.REGRESSION, **best_hp)
    best_model.compile(metrics=["mse"])
    best_model.fit(train_ds)

    rmse = math.sqrt(best_model.evaluate(test_ds, return_dict=True, verbose=0)["mse"])
    print(f"{mn}: RMSE: {rmse}, HP MRMSE: {mean_rmse}, HP SMRMSE: {std_rmse}")


Tuning: 0it [00:00, ?it/s]/tmp/ipykernel_9376/4074709627.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_fold_train["target"] = y_fold_train
/tmp/ipykernel_9376/4074709627.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_fold_test["target"] = y_fold_test
2025-03-29 08:05:02.166406: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Use /tmp/tmp_cxgckse as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.556848. Found 2257 examples.
Training model...


I0000 00:00:1743249902.751999    9376 kernel.cc:782] Start Yggdrasil model training
I0000 00:00:1743249902.752022    9376 kernel.cc:783] Collect training examples
I0000 00:00:1743249902.752027    9376 kernel.cc:795] Dataspec guide:
column_guides {
  column_name_pattern: "^__LABEL$"
  type: NUMERICAL
}
column_guides {
  column_name_pattern: "^dw_00$"
}
column_guides {
  column_name_pattern: "^dw_01$"
}
column_guides {
  column_name_pattern: "^dw_02$"
}
column_guides {
  column_name_pattern: "^dw_03$"
}
column_guides {
  column_name_pattern: "^dw_04$"
}
column_guides {
  column_name_pattern: "^dw_05$"
}
column_guides {
  column_name_pattern: "^dw_06$"
}
column_guides {
  column_name_pattern: "^dw_07$"
}
column_guides {
  column_name_pattern: "^dw_08$"
}
column_guides {
  column_name_pattern: "^dw_09$"
}
column_guides {
  column_name_pattern: "^dw_10$"
}
column_guides {
  column_name_pattern: "^dw_11$"
}
column_guides {
  column_name_pattern: "^psa_00$"
}
column_guides {
  column_name_pat